In [ ]:
import os
import requests

In [ ]:
%load_ext dotenv
%dotenv keys.env

api_key = os.getenv('bibKey')

In [ ]:
""" Enter the MMS Id of the bib you want to query, and then the 
holding ids of the holdings you want to query, separated by commas and enclosed by single quotes."""

mms_id = ''                  
holding_ids = ['', '', '']      

In [ ]:
def build_db(url):
   
    items = []
     
    total_record_count = float('inf')  # Initialize with a large value

    headers = {
        'apiKey': api_key, 
        'format': 'json',
        'limit' : 100,
        'offset' : 0
    }

    while headers['offset'] < total_record_count:     
            try:
                response = requests.get(url, params=headers)
            
                item_object = response.json()
                
                items.append(item_object)

                # Increment the offset by the 'limit' for the next page
                headers['offset'] += 100

                # Update total_record_count if it's not already set
                if 'total_record_count' in item_object and total_record_count == float('inf'):
                    total_record_count = item_object['total_record_count']

                # If we have reached the total count, exit the loop
                if headers['offset'] >= total_record_count:
                    break
            except Exception as e:
                # Handle other exceptions (e.g., network issues)
                print(f'{e} for {url}')
                break  # Exit the loop on exception

    return items

In [ ]:
# Creates the URL endpoints needed to make citation requests.
def build_url(holding_ids, mms_id): 
    a = []
    for i in holding_ids:
        holding_id = i

        url = f"https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/{mms_id}/holdings/{holding_id}/items"

        a.append(url)

    return a

In [ ]:
def unpack_values(response_list):
    values = []
    for i in response_list:
        for j in i['item']:
            values.append(j)
                       
    return values

In [ ]:
def get_counts(unpacked): 
        
    temp_location_count = len([i for i in unpacked if i['holding_data']['in_temp_location'] == True])
    base_status_0_count = len([i for i in unpacked if i['item_data']['base_status']['value'] == '0'])
    base_status_1_count = len([i for i in unpacked if i['item_data']['base_status']['value'] == '1'])
    holding_id = unpacked[0]['holding_data']['holding_id']
    location = unpacked[0]['item_data']['location']['desc']
    material_type = unpacked[0]['item_data']['physical_material_type']['value']
    base_status_0_process_types = {}

    for i in unpacked:
        if i['item_data']['base_status']['value'] == '0':
            process_type = i['item_data']['process_type']['value']
            if process_type not in base_status_0_process_types:
                base_status_0_process_types[process_type] = 1
            else:
                process_count = base_status_0_process_types.get(process_type, 0)
                process_count += 1
                base_status_0_process_types[process_type] = process_count

  
    a =  {  'holding_id': holding_id,
                'location' : location,
                'items_in_place': base_status_1_count,
                'items_not_in_place': base_status_0_count,
                'items_nip_process': base_status_0_process_types,
                'temp_location_count': temp_location_count, 
                'material_type': material_type
                }
    
    
    print(f'Inventory counts for Holding ID: {a["holding_id"]} in {a["location"]} with material type: {a["material_type"]}') 
    print(f'Available for checkout: {a["items_in_place"] - a["temp_location_count"]}')
    print(f'In temp location: {a["temp_location_count"]}')
    print(f'Items not in place: {a["items_not_in_place"]}, with process types: {a["items_nip_process"]}')
    print('\n')
    
    

In [ ]:
urls = build_url(holding_ids, mms_id) 

In [ ]:
for url in urls:
    item_responses = build_db(url)
    unpacked = unpack_values(item_responses)
    get_counts(unpacked)
